# LangGraph Stream Parser - Jupyter Display Demo

This notebook demonstrates the value of `JupyterDisplay` for visualizing LangGraph agent streams.

## The Problem

When streaming from a LangGraph agent, the raw output is:
- **Verbose** - Nested dictionaries with metadata you don't care about
- **Hard to follow** - Tool calls, messages, and interrupts all mixed together
- **Not real-time friendly** - Output floods the cell instead of updating in place

## The Solution

`JupyterDisplay` provides:
- **Clean panels** - Content, tools, and interrupts in organized boxes
- **Live updates** - Updates in place instead of flooding output
- **Tool lifecycle** - See tools transition from running → success/error with timing
- **Prominent interrupts** - Human-in-the-loop requests are impossible to miss

## Setup

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from agent import agent

---
## Before: Raw Streaming Output

This is what you get with vanilla `graph.stream()` - a flood of nested dictionaries:

In [3]:
# Raw streaming - hard to follow!
for chunk in agent.stream(
    dict(messages=["What files are in the current directory?"]), 
    config=dict(configurable=dict(thread_id="raw-demo")),
    stream_mode="updates"
):
    print(chunk)
    print("---")

{'PatchToolCallsMiddleware.before_agent': {'messages': Overwrite(value=[HumanMessage(content='What files are in the current directory?', additional_kwargs={}, response_metadata={}, id='20a3e7f9-3a16-4815-989b-4a66b708e36e')])}}
---
{'SummarizationMiddleware.before_model': None}
---{'model': {'messages': [AIMessage(content=[{'id': 'toolu_01DZjtuk3oQq3f1EUYnn6kmU', 'input': {'path': '/'}, 'name': 'ls', 'type': 'tool_use'}], additional_kwargs={}, response_metadata={'id': 'msg_01SiGGbADqXfqRHX3CfsumjB', 'model': 'claude-sonnet-4-5-20250929', 'stop_reason': 'tool_use', 'stop_sequence': None, 'usage': {'cache_creation': {'ephemeral_1h_input_tokens': 0, 'ephemeral_5m_input_tokens': 6258}, 'cache_creation_input_tokens': 6258, 'cache_read_input_tokens': 0, 'input_tokens': 3, 'output_tokens': 51, 'server_tool_use': None, 'service_tier': 'standard'}, 'model_name': 'claude-sonnet-4-5-20250929', 'model_provider': 'anthropic'}, name='Example', id='lc_run--019c1b36-ca44-73e1-9352-4dec64fddbda-0', too

---
## After: JupyterDisplay

Same stream, but with clean, live-updating panels:

In [12]:
from langgraph_stream_parser.adapters.jupyter import JupyterDisplay

In [11]:
display = JupyterDisplay()

display.stream(
    agent.stream(
        dict(messages=["What files are in the current directory?"]), 
        config=dict(configurable=dict(thread_id="display-demo")),
        stream_mode="updates"
    )
)

TOOL: [green]OK[/green] ls path=/ (3ms)

ASSISTANT: The current directory contains:

1. **/.ipynb_checkpoints/** - Directory (Jupyter checkpoint files)
2. **/__pycache__/** - Directory (Python cache files)
3. **/agent.py** - Python file
4. **/jupyter_display.py** - Python file
5. **/jupyter_example.ipynb** - Jupyter notebook file

So to answer your original question - would you like me to show you what's in the third item, which 
is `/agent.py`?

Done

In [5]:
stop

NameError: name 'stop' is not defined

In [13]:
display = JupyterDisplay()

display.stream(
    agent.stream(
        dict(messages=["what's in the third one?"]), 
        config=dict(configurable=dict(thread_id="display-demo")),
        stream_mode="updates"
    )
)

TOOL: [green]OK[/green] read_file file_path=/agent.py (5ms)

ASSISTANT: The third file, **/agent.py**, contains Python code that sets up a deep agent system. 
Here's what it does:

**Key components:**

1. **Imports**: Uses `deepagents`, `langgraph`, and standard Python libraries

2. **`think_tool` function** (lines 9-21): A reflection tool that allows the agent to pause and 
think about its actions

3. **`bash` function** (lines 24-88): Executes bash commands with:
   - Command execution in the workspace directory
   - Timeout handling (default 60 seconds)
   - Returns stdout, stderr, return code, and status
   - Error handling for timeouts and exceptions

4. **Agent setup** (lines 90-97):
   - Creates a `FilesystemBackend` with virtual mode enabled
   - Creates a deep agent named "Example" with:
     - The filesystem backend
     - Both `think_tool` and `bash` as available tools
     - Interrupts enabled for bash commands
     - In-memory checkpointing

This appears to be the main configuration file for a DeepAgents-based AI agent with filesystem 
access and bash execution capabilities.

Done

---
## Key Features

### 1. Tool Lifecycle Tracking

Tools show their status in real-time:
- `⏳ ...` - Running
- `✅ OK` - Success (with execution time)
- `❌ ERR` - Error

### 2. Interrupt Handling

When the agent needs approval (like for `bash` commands), you get a prominent red panel showing:
- The tool requesting approval
- The arguments it wants to use
- Allowed decisions (approve/reject/edit)

### 3. Content Accumulation

Assistant responses stream smoothly in a blue panel instead of chunk-by-chunk spam.

---
## Manual Event Processing

For more control, you can process events individually:

In [8]:
from langgraph_stream_parser import StreamParser
from langgraph_stream_parser.events import InterruptEvent, ContentEvent, ToolCallStartEvent

parser = StreamParser()
display = JupyterDisplay()

for event in parser.parse(
    agent.stream(
        dict(messages=["Count Python files in this directory"]),
        config=dict(configurable=dict(thread_id="manual-demo")),
        stream_mode="updates"
    )
):
    display.update(event)
    
    # You can also handle events programmatically
    if isinstance(event, InterruptEvent):
        print(f"\n[Interrupt detected - {len(event.action_requests)} action(s) need approval]")

TOOL: [green]OK[/green] ls path=/ (4ms)

TOOL: [green]OK[/green] glob pattern=**/*.py, path=/ (5ms)

ASSISTANT: I'll help you count the Python files in this directory. Let me start by listing the 
contents to see what we're working with.
Now let me search for all Python files recursively in this directory and its subdirectories:**Count:
3 Python files**

Here are the Python files found in this directory:
1. `/agent.py`
2. `/jupyter_display.py`
3. `/.ipynb_checkpoints/agent-checkpoint.py` (checkpoint file)

If you only want to count the main Python files (excluding checkpoint directories), there are **2 
Python files** (agent.py and jupyter_display.py).

Done

---
## Configuration Options

In [9]:
# Customize the display
display = JupyterDisplay(
    show_timestamps=False,     # Show/hide timestamps
    show_tool_args=True,       # Show tool arguments in the status table
    max_content_preview=200,   # Max characters for extraction previews
)

---
## Summary

| Raw Streaming | JupyterDisplay |
|---------------|----------------|
| Nested dicts flood output | Clean, organized panels |
| Tool status buried in data | Visual tool lifecycle (⏳→✅/❌) |
| Interrupts easy to miss | Prominent red interrupt panel |
| Content comes in chunks | Smooth content accumulation |
| No timing information | Execution time per tool |